<a href="https://colab.research.google.com/github/jylee2930/DataMining/blob/main/12%EC%A3%BC%EC%B0%A8_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
한국어 영화 리뷰 감정 분석 (NSMC Dataset)
Naver Sentiment Movie Corpus를 사용한 긍정/부정 분류 모델
"""

# ============================================================================
# 1단계: 라이브러리 임포트
# ============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import urllib.request
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ============================================================================
# 2단계: 데이터 다운로드 및 로드
# ============================================================================
print("=" * 50)
print("데이터 다운로드 중...")
print("=" * 50)

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
    filename="ratings_train.txt"
)
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
    filename="ratings_test.txt"
)

train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

print(f"\n학습 데이터 크기: {train_df.shape}")
print(f"테스트 데이터 크기: {test_df.shape}")
print(f"\n학습 데이터 샘플:\n{train_df.head()}")
print(f"\n레이블 분포:\n{train_df['label'].value_counts()}")

In [ ]:
# ============================================================================
# 3단계: Mecab 형태소 분석기 설치 (Google Colab)
# ============================================================================
print("\n" + "=" * 50)
print("Mecab 형태소 분석기 설치 중...")
print("=" * 50)

# Colab에서 실행
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab_light_220429.sh
%cd ..

from konlpy.tag import Mecab
mecab = Mecab()


In [ ]:
# ============================================================================
# 4단계: 데이터 전처리
# ============================================================================
print("\n" + "=" * 50)
print("데이터 전처리 시작...")
print("=" * 50)

# 한글 불용어 정의
STOP_WORDS = [
    "는", "을", "를", "이", "가", "의", "던", "고", "하", "다",
    "은", "에", "들", "지", "게", "도", "와", "과", "으로", "만"
]

def preprocess_text(text):
    """텍스트 전처리 함수"""
    if pd.isna(text):
        return []
    # 한글, 영어, 공백만 남기기
    text = re.sub("[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]", "", text)
    # 형태소 분석 및 불용어 제거
    tokens = mecab.morphs(text)
    tokens = [word for word in tokens if word not in STOP_WORDS and len(word) > 1]
    return tokens

# 학습 데이터 전처리
print("학습 데이터 전처리 중...")
train_df['document'] = train_df['document'].astype(str)
train_df = train_df.dropna(subset=['document'])
train_texts = train_df['document'].apply(preprocess_text)
train_labels = train_df['label'].values

# 테스트 데이터 전처리
print("테스트 데이터 전처리 중...")
test_df['document'] = test_df['document'].astype(str)
test_df = test_df.dropna(subset=['document'])
test_texts = test_df['document'].apply(preprocess_text)
test_labels = test_df['label'].values

# 빈 리뷰 제거
train_mask = train_texts.apply(lambda x: len(x) > 0)
train_texts = train_texts[train_mask]
train_labels = train_labels[train_mask]

test_mask = test_texts.apply(lambda x: len(x) > 0)
test_texts = test_texts[test_mask]
test_labels = test_labels[test_mask]

print(f"\n전처리 후 학습 데이터: {len(train_texts)}개")
print(f"전처리 후 테스트 데이터: {len(test_texts)}개")
print(f"샘플 전처리 결과: {train_texts.iloc[0]}")

In [ ]:
# ============================================================================
# 5단계: 토큰화 및 패딩
# ============================================================================
print("\n" + "=" * 50)
print("토큰화 및 시퀀스 변환...")
print("=" * 50)

# 개선: vocab_size 증가, OOV 토큰 사용
VOCAB_SIZE = 20000  # 15000 -> 20000으로 증가
MAX_LENGTH = 80      # 적절한 길이로 조정

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV>')
tokenizer.fit_on_texts(train_texts)

print(f"전체 단어 수: {len(tokenizer.word_index)}")
print(f"사용할 단어 수: {VOCAB_SIZE}")

# 시퀀스 변환
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# 패딩
train_padded = pad_sequences(train_sequences, maxlen=MAX_LENGTH,
                             padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=MAX_LENGTH,
                           padding='post', truncating='post')

print(f"패딩 후 shape: {train_padded.shape}")

In [ ]:
# ============================================================================
# 6단계: 개선된 모델 구축
# ============================================================================
print("\n" + "=" * 50)
print("모델 구축...")
print("=" * 50)

def create_improved_model():
    """개선된 양방향 LSTM 모델"""
    model = Sequential([
        # 임베딩 레이어 - 차원 증가
        Embedding(VOCAB_SIZE, 128, input_length=MAX_LENGTH),

        # 첫 번째 양방향 LSTM - return_sequences=True로 다층 구조
        Bidirectional(LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),

        # 두 번째 양방향 LSTM
        Bidirectional(LSTM(32, dropout=0.3, recurrent_dropout=0.3)),

        # Dense 레이어
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.3),

        # 출력 레이어
        Dense(1, activation='sigmoid')
    ])

    # Adam optimizer with custom learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    return model

model = create_improved_model()
model.summary()

In [ ]:
# ============================================================================
# 7단계: 콜백 설정 및 학습
# ============================================================================
print("\n" + "=" * 50)
print("모델 학습 시작...")
print("=" * 50)

# 콜백 설정
checkpoint_path = 'best_sentiment_model.weights.h5'

callbacks = [
    # 최고 성능 모델 저장
    ModelCheckpoint(
        checkpoint_path,
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
        verbose=1
    ),

    # 조기 종료
    EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),

    # 학습률 감소
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=0.00001,
        verbose=1
    )
]

# 모델 학습
history = model.fit(
    train_padded,
    train_labels,
    validation_split=0.2,  # 학습 데이터의 20%를 검증용으로 사용
    batch_size=128,         # 배치 크기 증가
    epochs=15,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# ============================================================================
# 8단계: 학습 결과 시각화
# ============================================================================
print("\n" + "=" * 50)
print("학습 결과 시각화...")
print("=" * 50)

def plot_training_history(history):
    """학습 이력 시각화"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # 정확도 그래프
    axes[0].plot(history.history['accuracy'], label='Train Accuracy')
    axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[0].set_title('Model Accuracy', fontsize=14)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True)

    # 손실 그래프
    axes[1].plot(history.history['loss'], label='Train Loss')
    axes[1].plot(history.history['val_loss'], label='Validation Loss')
    axes[1].set_title('Model Loss', fontsize=14)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_training_history(history)

In [ ]:
# ============================================================================
# 9단계: 테스트 데이터 평가
# ============================================================================
print("\n" + "=" * 50)
print("테스트 데이터 평가...")
print("=" * 50)

# 최고 성능 가중치 로드
model.load_weights(checkpoint_path)

# 평가
test_loss, test_acc, test_precision, test_recall = model.evaluate(
    test_padded,
    test_labels,
    verbose=1
)

print(f"\n최종 테스트 결과:")
print(f"정확도: {test_acc:.4f}")
print(f"정밀도: {test_precision:.4f}")
print(f"재현율: {test_recall:.4f}")
print(f"F1-Score: {2 * (test_precision * test_recall) / (test_precision + test_recall):.4f}")

In [ ]:
# ============================================================================
# 10단계: 예측 함수
# ============================================================================
print("\n" + "=" * 50)
print("예측 함수 준비 완료")
print("=" * 50)

def predict_sentiment(text, show_detail=True):
    """
    새로운 텍스트의 감정을 예측하는 함수

    Args:
        text (str): 예측할 텍스트
        show_detail (bool): 상세 정보 출력 여부

    Returns:
        tuple: (예측 확률, 감정 레이블)
    """
    # 전처리
    processed = preprocess_text(text)

    if len(processed) == 0:
        return 0.5, "중립"

    # 시퀀스 변환 및 패딩
    sequence = tokenizer.texts_to_sequences([processed])
    padded = pad_sequences(sequence, maxlen=MAX_LENGTH, padding='post', truncating='post')

    # 예측
    prediction = model.predict(padded, verbose=0)[0][0]

    # 결과 해석
    sentiment = "긍정" if prediction > 0.5 else "부정"
    confidence = prediction if prediction > 0.5 else (1 - prediction)

    if show_detail:
        print(f"\n입력 텍스트: {text}")
        print(f"전처리 결과: {processed}")
        print(f"예측 점수: {prediction:.4f}")
        print(f"감정: {sentiment} ({confidence*100:.2f}% 확신)")

    return prediction, sentiment

In [ ]:
# ============================================================================
# 11단계: 예측 테스트
# ============================================================================
print("\n" + "=" * 50)
print("예측 테스트")
print("=" * 50)

# 테스트 샘플
test_samples = [
    "이 영화 진짜 최고였어요! 강추합니다!",
    "완전 시간 낭비 ㅉㅉ 돈 아까워",
    "와 개쩐다 정말 세계관 최강자들의 영화다",
    "이게 영화야? 차라리 유튜브 보는게 나음",
    "배우들 연기가 너무 좋았고 스토리도 탄탄했어요",
    "졸려서 중간에 잤음 ㅋㅋ",
    "기대 안했는데 의외로 괜찮네요",
    "역대급 쓰레기 영화"
]

for sample in test_samples:
    predict_sentiment(sample)
    print("-" * 50)

In [ ]:
# ============================================================================
# 12단계: 모델 저장
# ============================================================================
print("\n" + "=" * 50)
print("모델 저장...")
print("=" * 50)

# 전체 모델 저장
model.save('sentiment_model_full.h5')
print("모델이 'sentiment_model_full.h5'에 저장되었습니다.")

# 토크나이저 저장 (pickle 사용)
import pickle
with open('tokenizer.pickle', 'wb') as f:
    pickle.dump(tokenizer, f)
print("토크나이저가 'tokenizer.pickle'에 저장되었습니다.")

print("\n" + "=" * 50)
print("모든 작업이 완료되었습니다!")
print("=" * 50)
